# Libraries

In [1]:
import pickle
from sklearn.metrics import log_loss
from __future__ import print_function

# Get Data & Models For Testing

In [2]:
path = "/Users/davidziganto/Repositories/Synthetic_Dataset_Generation/pickle_files/py35/"

with open(path + "X_test_py35.pkl", 'rb') as picklefile: 
    X_test = pickle.load(picklefile)
    
with open(path + "y_test_py35.pkl", 'rb') as picklefile: 
    y_test = pickle.load(picklefile)

with open(path + "knn_needs_improvement_py35.pkl", 'rb') as picklefile: 
    knn_needs_improvement = pickle.load(picklefile)
    
with open(path + "rf_satisfactory_py35.pkl", 'rb') as picklefile: 
    rf_satisfactory = pickle.load(picklefile)

with open(path + "gbc_proficient_py35.pkl", 'rb') as picklefile: 
    gbc_proficient = pickle.load(picklefile)

# Calculate Log Loss

In [3]:
log_loss(y_test, knn_needs_improvement.predict_proba(X_test))

0.75208153985645276

In [4]:
log_loss(y_test, rf_satisfactory.predict_proba(X_test))

0.53246821862725746

In [5]:
log_loss(y_test, gbc_proficient.predict_proba(X_test))

0.51749469915046675

# Create Pickle_Dict

In [6]:
pickle_dict = {'knn':knn_needs_improvement, 'rf_':rf_satisfactory, 'gbc':gbc_proficient}

# Auto_Score()

In [7]:
def auto_score(pickle_dict):
    '''
    Input:
        pickle_dict: dictionary where key is username | ID and value is model
    Output:
        username : [log loss value, classification] 
    '''
    for k, v in pickle_dict.items():
        score = log_loss(y_test, v.predict_proba(X_test))
        if score < 0.530:
            pickle_dict[k] = [score, "Proficient"]
        elif score <= 0.540:
            pickle_dict[k] = [score, "Satisfactory"]
        else:
            pickle_dict[k] = [score, "Needs Improvement"]
            
    return pickle_dict

In [8]:
output = auto_score(pickle_dict)
output

{'gbc': [0.51749469915046675, 'Proficient'],
 'knn': [0.75208153985645276, 'Needs Improvement'],
 'rf_': [0.53246821862725746, 'Satisfactory']}

In [9]:
for k,v in output.items():
    print(k, output[k][0], output[k][1])

gbc 0.51749469915 Proficient
rf_ 0.532468218627 Satisfactory
knn 0.752081539856 Needs Improvement


# Rationale For Scoring

Default settings for Decision Trees, KNN, Logistic Regression, Multinomial Naive Bayes, and Random Forest yield log loss values on the test set of:

|Algo|Log Loss|Type|
|---|---|---|
|DT|12.676|Std|
|GBC|0.517|Adv|
|KNN|2.064|Std|
|LR|0.543|Std|
|NB|0.963|Std|
|RF|0.566|Std|


**Std=standard algorithm**; 
**Adv=advanced algorithm**

Tuned versions yield the following log loss values:

|Algo|Log Loss|Type|
|---|---|---|
|DT|0.797|Std|
|GBC|0.517|Adv|
|KNN|0.752|Std|
|LR|0.543|Std|
|NB|0.963|Std|
|RF|0.532|Std|

The goal here is to determine each invidual's skill level in achieving performant modeling results. As such, it made sense to set the threshold for *satisfactory* just below the lowest log loss value yielded by the default model settings, which is logistic regression at 0.543. Therefore, the threshold is set at 0.540.

Furthermore, in an attempt to separate the high-achieving students, a category called *proficient* is included. In order to achieve this status, a student must use modeling techniques either not covered or covered only in very little detail to achieve the required log loss value. Therefore, the threshold (0.530) was set just below the log loss value of the lowest log loss value for the 'stock' algorithms, which is tuned random forest (0.532). For instance, a tuned gradient boosted classifier can achieve a log loss of 0.517.

**NOTES**   
1. There will be some variablity with inherently non-deterministic algorithms like random forest. The techniques will be the same but the results may vary slightly due to how the model was seeded.  
2.  This is a first attempt. Score thresholds can be adjusted as we collect data in our pilot program. In other words, this is a WIP.